In [2]:
import pandas as pd
import numpy as np
import concurrent.futures as cf
import datetime as dt

In [38]:
# here I change the type in case of numerical values being str
def change_type(df,n=0):
    col=df.columns[n]
    
    if df[col].dtype=="O":
            try:
               df=df.astype({col:"float64"})
               
            except:
                df=df.astype({col:"str"})
                
                 
            
    if n==len(df.columns)-1:
        return df
    else:
        n+=1
        return change_type(df,n)


In [2]:
# here I create 2 columns and define all time columns 
def clean_col(df):
    
    df["S_2"]=df["S_2"].astype('datetime64[ns]')
    df["Year"]=df["S_2"].dt.year
    df["Month"]=df["S_2"].dt.month

    return df
            


In [3]:
# here I make  a list of the permutation of each dataframe in the list
def reage_list(long,n=0,new_lis1=[],new_lis2=[]):
    
    new_lis1=list(map(lambda x: [long[n],x],long))

    new_lis2=new_lis2+new_lis1

    if n==len(long)-1:

        new_lis2=list(filter(lambda x:x  if x[0]!=x[1] else None,new_lis2))

        return new_lis2

    else:

        n+=1

        return reage_list(long,n,new_lis1,new_lis2)

In [4]:
# here I concatonate all the customer_ID into the same data frame and drop that value from the prior dataframe
def organise_df(data,inter,n=0):
    now=inter[n]
    try: 
    
        data[now[0][1]]=pd.concat([data[now[0][1]],data[now[0][0]][data[now[0][0]]["customer_ID"]==now[1][0]]])
    
        data[now[0][0]]=data[now[0][0]].drop(data[now[0][0]][data[now[0][0]]["customer_ID"]==now[1][0]].index)
        
    except:
        pass

    if n==len(inter)-1:
        return data
    else:
        n+=1
        return organise_df(data,inter,n)

In [5]:
def auto_sys(csv):

    chunks=pd.read_csv(csv,chunksize=500000)

    with cf.ThreadPoolExecutor() as executor:
        data=list(executor.map(lambda x:x , chunks))

    arr=np.arange(len(data)).tolist()
    
    arr_lis=reage_list(arr)

    # here I check using the permutation list the interseption that exist between dataframes in customer_ID 
    with cf.ThreadPoolExecutor() as executor:
        inter=list(executor.map(lambda x:[x, np.intersect1d(data[x[0]]["customer_ID"].unique(), (data[x[1]]["customer_ID"].unique())).tolist()] ,arr_lis))
    

    data=organise_df(data,inter)



    with cf.ThreadPoolExecutor() as executor:
        data=list(executor.map(clean_col,data))
    

    
    return data


In [6]:
lista_df=auto_sys("train_data.csv")

In [ ]:
#save each obj in the list to csv

In [8]:
def save_csv(lista_df,n=0,lista_link=[]):
    
    link="train_num"+str(n)+".csv"

    df=lista_df[n]
    
    df.to_csv(link)

    lista_link.append(link)
    
    if n==len(lista_df)-1:
        
        return lista_link

    else:

        n+=1

        return save_csv(lista_df,n,lista_link)
    

In [9]:
lista_link=save_csv(lista_df)

In [10]:
lista_link

['train_num0.csv',
 'train_num1.csv',
 'train_num2.csv',
 'train_num3.csv',
 'train_num4.csv',
 'train_num5.csv',
 'train_num6.csv',
 'train_num7.csv',
 'train_num8.csv',
 'train_num9.csv',
 'train_num10.csv',
 'train_num11.csv']

In [11]:
del lista_df

In [2]:
# here I map out all the different month data that exists per Dataframe and had that to a dicionary where the key is the number of months
def map_data(data,gr_data,num_data,n=0,dic={}):

    

    array_ID=gr_data[gr_data["S_2"]==num_data[n]]["customer_ID"].unique()

    new_frame=data[data["customer_ID"].isin(array_ID)]

    dic.update({num_data[n]:new_frame})

    if n==len(num_data)-1:
        return dic
    else:
        n+=1
        return map_data(data,gr_data,num_data,n,dic)


In [3]:
# here I make a dicionary to group data by month per customer_ID
def dic_maker(list_link):

    data=pd.read_csv(list_link)
    
    gr_data=data.groupby("customer_ID")["S_2"].count().to_frame().reset_index()

    num_data=list(gr_data["S_2"].unique())

    max_val=max(num_data)
    
    num_data.pop(num_data.index(max_val))
    
    n=0
    
    dic={}
    
    dic=map_data(data,gr_data,num_data,n,dic)
    
    array_ID=gr_data[gr_data["S_2"]==max_val]["customer_ID"].unique()
    
    data=data[data["customer_ID"].isin(array_ID)]
    
    dic.update({max_val:data})

    del data

    return dic

In [4]:
lista_link=['train_num0.csv',
 'train_num1.csv',
 'train_num2.csv',
 'train_num3.csv',
 'train_num4.csv',
 'train_num5.csv',
 'train_num6.csv',
 'train_num7.csv',
 'train_num8.csv',
 'train_num9.csv',
 'train_num10.csv',
 'train_num11.csv']

In [5]:
with cf.ThreadPoolExecutor() as excutor:
        data=list(excutor.map(dic_maker,lista_link))


In [6]:
# this is a funcion to delet entry in the dicionary as they are being group by month
def del_dic(data,num_key,num):

    del data[num][num_key]

    if num==len(data)-1:
        return data
    else:
        num+=1
        return del_dic(data,num_key,num)

In [7]:
key_list=list(data[0].keys())
arr_key=list(np.arange(1,max(key_list)))

In [8]:
# this his how i concatonate all entries by month together with the exeption of the 13 month data
def concat_dic(data,arr_key):

    num_key=arr_key[0]

    data[0][num_key]=pd.concat(list(map(lambda x : x[num_key] , data)))

    num=1
    
    data=del_dic(data,num_key,num)
    
    arr_key.pop(0)

    if len(arr_key)==0:
        return data

    else:
        return concat_dic(data,arr_key)

In [9]:
data=concat_dic(data,arr_key)

In [10]:
# this is the number of unique customer by month data 
for x in range(1,len(data[0].keys())+1):
    print(x)
    print(len(data[0][x]["customer_ID"].unique()))

1
5120
2
6098
3
5778
4
4673
5
4671
6
5515
7
5198
8
6110
9
6411
10
6721
11
5961
12
10623
13
34858


In [13]:
# here I save the dataframes as csv from the dicionary
def read_dic(data,data_keys,n=0):

    df=data[data_keys[0]]

    if len(data_keys)==1:
        n+=1
        link="train_month"+str(data_keys[0])+"_"+str(n)+".csv"

    else:
        link="train_month"+str(data_keys[0])+".csv"
    
    df.to_csv(link)

    data_keys.pop(0)


    if len(data_keys)==0:
        
        del data

        return 

    else:

        return read_dic(data,data_keys,n)
    
    

In [18]:
data=list(map(lambda x: read_dic(data=x,data_keys=list(x.keys())),data))

In [19]:
# this is for the 13 month data
def save_me(data,n=0):
    df=data[n][13]

    link="train_month13_"+str(n)+".csv"

    df.to_csv(link)

    if n==len(data)-1:
        return

    else:
        n+=1
        return save_me(data,n)
    


In [20]:
save_me(data)

In [49]:
link_list=["train_month1.csv","train_month2.csv","train_month3.csv",
"train_month4.csv","train_month5.csv","train_month6.csv",
"train_month7.csv","train_month8.csv","train_month9.csv",
"train_month10.csv","train_month11.csv","train_month12.csv",
"train_month13_0.csv","train_month13_1.csv","train_month13_2.csv",
"train_month13_3.csv","train_month13_4.csv","train_month13_5.csv",
"train_month13_6.csv","train_month13_7.csv","train_month13_8.csv",
"train_month13_10.csv","train_month13_11.csv"]

In [50]:
# here I take out all columns with more then 75 of NAN
def change_col(link):
    
    data_1=pd.read_csv(link)
    
    data_1=change_type(data_1)
    
    lista_col=list(filter(lambda x: x if (data_1[x].isnull().sum()/len(data_1[x]))*100>=75 else None,data_1.columns))

    data_1.drop(labels=lista_col,axis=1,inplace=True)

    data_1.drop(labels=['Unnamed: 0.1', 'Unnamed: 0'],axis=1,inplace=True)

    data_1.to_csv(link)
    
    return


In [54]:
with cf.ThreadPoolExecutor() as excutor:
    list(excutor.map(change_col,link_list))